<a href="https://colab.research.google.com/github/J-Bourgeais/AI_TDDC17_LiU/blob/main/planning_lab_stub_sansA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Planning Lab

In [ ]:
!pip install --pre unified-planning==1.0.0.95.dev1
!pip install up_fast_downward==0.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.0/643.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 17.6 MB/s eta 0:00:00


In [ ]:
!pip install matplotlib==3.7.1  # for visualisation in this notebook

In [ ]:
from unified_planning.shortcuts import *

import unified_planning as up

up.shortcuts.get_environment().credits_stream = None

## Part (a): Model the Task

The code below models a simpler version of the Household Robot domain with two rooms and one open door between them. The robot is in room A initially and needs to move to room B. You can use it as a starting point for your own solution to the full Household Robot task.

In [ ]:

def get_planning_task():
    # Declare user types.
    Room = UserType("Room")
    Door = UserType("Door")
    Key = UserType("Key")

    # Declare predicates.
    robot_in = up.model.Fluent("robot_in", BoolType(), r=Room)
    key_in = up.model.Fluent("key_in", BoolType(), k=Key, r=Room)
    robot_has_key = up.model.Fluent("robot_has_key", BoolType(), k=Key)
    door_is_open = up.model.Fluent("door_is_open", BoolType(), d=Door)
    connected = up.model.Fluent("connected", BoolType(), r1=Room, d=Door, r2=Room)
    room_has_door = up.model.Fluent("room_has_door", BoolType(), r=Room, d=Door)
    #Maybe add predicates for everything key' related

    # Add (typed) objects to problem.
    problem = up.model.Problem("household")

    def get_room(room):
        return up.model.Object(f"room{room}", Room)

    def get_door(door):
        return up.model.Object(f"door{door}", Door)

    def get_key(key):
        return up.model.Object(f"key{key}", Key)

    Kitchen = get_room("Kitchen")
    LivingRoom = get_room("LivingRoom")
    Corridor = get_room("Corridor")
    Lobby = get_room("Lobby")
    Bathroom = get_room("Bathroom")
    Outside = get_room("Outside")
    rooms = [Kitchen, LivingRoom, Corridor, Bathroom, Lobby, Outside]

    doorK = get_door("K")
    doorL = get_door("L")
    doorB = get_door("B")
    doorC = get_door("C")
    doorF = get_door("F")
    doors = [doorK, doorL, doorB, doorC, doorF]

    keyK = get_key("K");
    keyL = get_key("L");
    keyB = get_key("B");
    keyC = get_key("C");
    keyF = get_key("F");
    keyAll = get_key("All");
    keys = [keyK, keyL, keyB, keyC, keyF, keyAll]

    problem.add_objects(rooms)
    problem.add_objects(doors)
    problem.add_objects(keys)

    connections = [
        (Kitchen, doorK, LivingRoom), (LivingRoom, doorL, Corridor), (Corridor, doorB, Bathroom), (Corridor, doorC, Lobby), (Lobby, doorF, Outside)
    ]

    rooms_and_doors = []
    for room1, door, room2 in connections:
        rooms_and_doors.append((room1, door))
        rooms_and_doors.append((room2, door))

    # Specify the initial state.
    problem.add_fluent(robot_in, default_initial_value=False)
    problem.add_fluent(connected, default_initial_value=False)
    problem.add_fluent(room_has_door, default_initial_value=False)
    problem.add_fluent(key_in, default_initial_value=False)
    problem.add_fluent(robot_has_key, default_initial_value=False)
    problem.add_fluent(door_is_open, default_initial_value=False)

    #add for the others
    problem.set_initial_value(robot_in(LivingRoom), True)
  #Set where keys are
    problem.set_initial_value(key_in(keyK, LivingRoom), True)
    problem.set_initial_value(key_in(keyL, LivingRoom), True)
    problem.set_initial_value(key_in(keyB, Corridor), True)
    problem.set_initial_value(key_in(keyF, Bathroom), True)
    problem.set_initial_value(key_in(keyC, Bathroom), True)
    problem.set_initial_value(key_in(keyAll, Kitchen), True)

    for room1, door, room2 in connections:
        problem.set_initial_value(connected(room1, door, room2), True)
        problem.set_initial_value(connected(room2, door, room1), True)
    for room, door in rooms_and_doors:
        problem.set_initial_value(room_has_door(room, door), True)

    # Add actions

#MOVE BETWEEN ROOM
    #Move from Kitchen to LivingRoom
    move_K_LR = up.model.InstantaneousAction("move_K_LR")
    move_K_LR.add_precondition(robot_in(Kitchen))
    move_K_LR.add_precondition(connected(Kitchen, doorK, LivingRoom))
    move_K_LR.add_precondition(door_is_open(doorK))
    move_K_LR.add_effect(robot_in(Kitchen), False)
    move_K_LR.add_effect(robot_in(LivingRoom), True)
    problem.add_action(move_K_LR)

    #Move from LivingRoom to Kitchen
    move_LR_K = up.model.InstantaneousAction("move_LR_K")
    move_LR_K.add_precondition(robot_in(LivingRoom))
    move_LR_K.add_precondition(connected(Kitchen, doorK, LivingRoom))
    move_LR_K.add_precondition(door_is_open(doorK))
    move_LR_K.add_effect(robot_in(Kitchen), True)
    move_LR_K.add_effect(robot_in(LivingRoom), False)
    problem.add_action(move_LR_K)

    #Move from LivingRoom to Corridor
    move_LR_C = up.model.InstantaneousAction("move_LR_C")
    move_LR_C.add_precondition(robot_in(LivingRoom))
    move_LR_C.add_precondition(connected(LivingRoom, doorL, Corridor))
    move_LR_C.add_precondition(door_is_open(doorL))
    move_LR_C.add_effect(robot_in(LivingRoom), False)
    move_LR_C.add_effect(robot_in(Corridor), True)
    problem.add_action(move_LR_C)

    #Move from Corridor to LivingRoom
    move_C_LR = up.model.InstantaneousAction("move_C_LR")
    move_C_LR.add_precondition(robot_in(Corridor))
    move_C_LR.add_precondition(connected(LivingRoom, doorL, Corridor))
    move_C_LR.add_precondition(door_is_open(doorL))
    move_C_LR.add_effect(robot_in(LivingRoom), True)
    move_C_LR.add_effect(robot_in(Corridor), False)
    problem.add_action(move_C_LR)

    #Move from Corridor to Bathroom
    move_C_B = up.model.InstantaneousAction("move_C_B")
    move_C_B.add_precondition(robot_in(Corridor))
    move_C_B.add_precondition(connected(Corridor, doorB, Bathroom))
    move_C_B.add_precondition(door_is_open(doorB))
    move_C_B.add_effect(robot_in(Corridor), False)
    move_C_B.add_effect(robot_in(Bathroom), True)
    problem.add_action(move_C_B)

    #Move from Bathroom to Corridor
    move_B_C = up.model.InstantaneousAction("move_B_C")
    move_B_C.add_precondition(robot_in(Bathroom))
    move_B_C.add_precondition(connected(Corridor, doorB, Bathroom))
    move_B_C.add_precondition(door_is_open(doorB))
    move_B_C.add_effect(robot_in(Corridor), True)
    move_B_C.add_effect(robot_in(Bathroom), False)
    problem.add_action(move_B_C)

    #Move from Corridor to Lobby
    move_C_L = up.model.InstantaneousAction("move_C_L")
    move_C_L.add_precondition(robot_in(Corridor))
    move_C_L.add_precondition(connected(Corridor, doorC, Lobby))
    move_C_L.add_precondition(door_is_open(doorC))
    move_C_L.add_effect(robot_in(Corridor), False)
    move_C_L.add_effect(robot_in(Lobby), True)
    problem.add_action(move_C_L)

    #Move from Lobby to Corridor
    move_L_C = up.model.InstantaneousAction("move_L_C")
    move_L_C.add_precondition(robot_in(Lobby))
    move_L_C.add_precondition(connected(Corridor, doorC, Lobby))
    move_L_C.add_precondition(door_is_open(doorC))
    move_L_C.add_effect(robot_in(Corridor), True)
    move_L_C.add_effect(robot_in(Lobby), False)
    problem.add_action(move_L_C)

#PICK_A_KEY
    #Pick the key K
    Pick_Key_K = up.model.InstantaneousAction("Pick_Key_K")
    Pick_Key_K.add_precondition(robot_in(LivingRoom))
    Pick_Key_K.add_precondition(key_in(keyK, LivingRoom))
    Pick_Key_K.add_effect(robot_has_key(keyK), True)
    Pick_Key_K.add_effect(key_in(keyK, LivingRoom), False)
    problem.add_action(Pick_Key_K)

    #faut qu'il l'ai pas deja

    #Pick the key L
    Pick_Key_L = up.model.InstantaneousAction("Pick_Key_L")
    Pick_Key_L.add_precondition(robot_in(LivingRoom))
    Pick_Key_L.add_precondition(key_in(keyL, LivingRoom))
    Pick_Key_L.add_effect(robot_has_key(keyL), True)
    Pick_Key_L.add_effect(key_in(keyL, LivingRoom), False)
    problem.add_action(Pick_Key_L)

    #Pick the key all
    Pick_Key_all = up.model.InstantaneousAction("Pick_Key_all")
    Pick_Key_all.add_precondition(robot_in(Kitchen))
    Pick_Key_all.add_precondition(key_in(keyAll, Kitchen))
    Pick_Key_all.add_effect(robot_has_key(keyAll), True)
    Pick_Key_all.add_effect(key_in(keyAll, Kitchen), False)
    problem.add_action(Pick_Key_all)

    #Pick the key B
    Pick_Key_B = up.model.InstantaneousAction("Pick_Key_B")
    Pick_Key_B.add_precondition(robot_in(Corridor))
    Pick_Key_B.add_precondition(key_in(keyB, Corridor))
    Pick_Key_B.add_effect(robot_has_key(keyB), True)
    Pick_Key_B.add_effect(key_in(keyB, Corridor), False)
    problem.add_action(Pick_Key_B)

    #Pick the key F
    Pick_Key_F = up.model.InstantaneousAction("Pick_Key_F")
    Pick_Key_F.add_precondition(robot_in(Bathroom))
    Pick_Key_F.add_precondition(key_in(keyF, Bathroom))
    Pick_Key_F.add_effect(robot_has_key(keyF), True)
    Pick_Key_F.add_effect(key_in(keyF, Bathroom), False)
    problem.add_action(Pick_Key_F)

    #Pick the key C
    Pick_Key_C = up.model.InstantaneousAction("Pick_Key_C")
    Pick_Key_C.add_precondition(robot_in(Bathroom))
    Pick_Key_C.add_precondition(key_in(keyC, Bathroom))
    Pick_Key_C.add_effect(robot_has_key(keyC), True)
    Pick_Key_C.add_effect(key_in(keyC, Bathroom), False)
    problem.add_action(Pick_Key_C)

#OPEN A DOOR
    #Open the door K
    Open_K = up.model.InstantaneousAction("Open_K")
    Open_K.add_precondition(robot_has_key(keyK).Or(robot_has_key(keyAll)))
    Open_K.add_precondition(robot_in(Kitchen).Or(robot_in(LivingRoom)))
    Open_K.add_effect(door_is_open(doorK), True)
    problem.add_action(Open_K)

    #Open the door L
    Open_L = up.model.InstantaneousAction("Open_L")
    Open_L.add_precondition(robot_has_key(keyL).Or(robot_has_key(keyAll)))
    Open_L.add_precondition(robot_in(LivingRoom).Or(robot_in(Corridor)))
    Open_L.add_effect(door_is_open(doorL), True)
    problem.add_action(Open_L)

    #Open the door B
    Open_B = up.model.InstantaneousAction("Open_B")
    Open_B.add_precondition(robot_has_key(keyB).Or(robot_has_key(keyAll)))
    Open_B.add_precondition(robot_in(Corridor).Or(robot_in(Bathroom)))
    Open_B.add_effect(door_is_open(doorB), True)
    problem.add_action(Open_B)

    #Open the door C
    Open_C = up.model.InstantaneousAction("Open_C")
    Open_C.add_precondition(robot_has_key(keyC).Or(robot_has_key(keyAll)))
    Open_C.add_precondition(robot_in(Corridor).Or(robot_in(Lobby)))
    Open_C.add_effect(door_is_open(doorC), True)
    problem.add_action(Open_C)

    #Open the door F
    Open_F = up.model.InstantaneousAction("Open_F")
    Open_F.add_precondition(robot_has_key(keyF).Or(robot_has_key(keyAll)))
    Open_F.add_precondition(robot_in(Lobby))
    Open_F.add_effect(door_is_open(doorF), True)
    problem.add_action(Open_F)


    # Specify the goal.
    problem.add_goal(door_is_open(doorF))

    # We want to minimize the plan cost.
    problem.add_quality_metric(MinimizeActionCosts({}, default=Int(1)))
    return problem

problem = get_planning_task()
#print(problem)


## Part (b): Find a (possibly suboptimal) plan

Solve the task with greedy best-first search using the FF heuristic. The example code below uses the h^add heuristic. You need to inspect the output to stdout to see the heuristic value of the initial state.

In [ ]:
params = {
    "fast_downward_search_config": "eager_greedy([add()])"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)
        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)
    else:
        print("No plan found.")

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmpowk_mfqf/domain.pddl /tmp/tmpowk_mfqf/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.000s CPU, 0.006s wall-clock]
Normalizing task... [0.000s CPU, 0.001s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.001s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 65 rules.
Computing model... [0.000s CPU, 0.001s wall-clock]
126 relevant atoms
0 auxiliary atoms
126 final queue length
143 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Insta

In [ ]:
params = {
    "fast_downward_search_config": "eager_greedy([ff()])"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)
        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)
    else:
        print("No plan found.")

INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmpwoxix9z0/domain.pddl /tmp/tmpwoxix9z0/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.000s CPU, 0.004s wall-clock]
Normalizing task... [0.000s CPU, 0.001s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.002s wall-clock]
Preparing model... [0.000s CPU, 0.002s wall-clock]
Generated 65 rules.
Computing model... [0.000s CPU, 0.001s wall-clock]
126 relevant atoms
0 auxiliary atoms
126 final queue length
143 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Insta

## Part (c): Find an optimal plan+

Solve the task with A* using the `iPDB` heuristic.

In [ ]:
params = {
    "fast_downward_search_config": "astar(ipdb())"
}

with OneshotPlanner(name="fast-downward", params=params) as planner:
    result = planner.solve(problem, output_stream=sys.stdout)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("Found a plan of length:", len(result.plan.actions))
        print(result.plan)
        with PlanValidator() as validator:
            val_result = validator.validate(problem, result.plan)
            print("Plan cost:", val_result.metric_evaluations)
    else:
        print("No plan found.")


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.
INFO     translator stdin: None
INFO     translator time limit: None
INFO     translator memory limit: None
INFO     translator command line string: /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/up_fast_downward/downward/builds/release/bin/translate/translate.py /tmp/tmpaes6zbls/domain.pddl /tmp/tmpaes6zbls/problem.pddl --sas-file output.sas
Parsing...
Parsing: [0.000s CPU, 0.004s wall-clock]
Normalizing task... [0.000s CPU, 0.001s wall-clock]
Instantiating...
Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.010s CPU, 0.001s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 65 rules.
Computing model... [0.000s CPU, 0.001s wall-clock]
126 relevant atoms
0 auxiliary atoms
126 final queue length
143 total queue pushes
Completing instantiation... [0.000s CPU, 0.001s wall-clock]
Insta